# Matt Sloane / ms9548
## Homework 11, Assignment 2

In [20]:
# Import modules / setup notebook environment
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import zipfile
import json
import fiona
from fiona.crs import from_epsg
import pylab as pl
import shapely
%pylab inline

s = json.load(open(os.getenv("PUIDATA")+'/fbb_matplotlibrc.json'))
pl.rcParams.update(s)   

pui_Data = os.getenv("PUIDATA")

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [21]:
def get_data():
    '''
    Download data for Homework 11, Assigment 2 and move to HW11 PUI Data Folder, including:
    1.  Census Data for Businesses by Zipcode
    2.  NYC Zipcodes Shapefile 
    '''
    
    # Download NYC Zipcode Shapefile from GitHub HW11 Repository
    
    url_base = "https://raw.githubusercontent.com/fedhere/PUI2016_fb55/master/HW11_fb55"
    !curl -O "{url_base}/nyc-zip-code-tabulation-areas-polygons.geojson"
    
    #  Commenting out this portion of the function to avoid redundant downloads due to long download times
    '''
    # Download Census Data for Business by Zip Code (Code provided by professor)

    !for ((y=93; y<=99; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done 
    !for ((y=0; y<=1; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done
    !for ((y=2; y<=9; y+=1)); do wget ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done
    !for ((y=10; y<=15; y+=1)); do wget ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done
    '''
    
get_data()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  591k  100  591k    0     0  1429k      0 --:--:-- --:--:-- --:--:-- 5977k


In [22]:
#create a new dataframe with values from all census business data files - from KDR276
years = ['94', '95', '96', '97', '98', '99', '00', '01', '02', '03', '04', '05', '06', '07',
        '08', '09', '10', '11', '12', '13', '14']
CombinedValues = pd.DataFrame()
for year in years:
    fname = 'zbp' + year + 'totals.zip'
    zf = zipfile.ZipFile(fname)
    each_year = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    each_year['year'] = year
    CombinedValues = pd.concat([CombinedValues, each_year], axis=0)
    
CombinedValues.head()

,AP,EMP,EMPFLAG,EST,NAME,QP1,ZIP,ap,ap_nf,city,...,emp,emp_nf,empflag,est,name,qp1,qp1_nf,stabbr,year,zip
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155158.0,NaN,NaN,...,6198.0,NaN,NaN,439.0,"Agawam, MA",33601.0,NaN,NaN,94,1001.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127367.0,NaN,NaN,...,6073.0,NaN,NaN,450.0,"Amherst, MA",28924.0,NaN,NaN,94,1002.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1604.0,NaN,NaN,...,68.0,NaN,NaN,10.0,"Amherst, MA",367.0,NaN,NaN,94,1003.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4302.0,NaN,NaN,...,210.0,NaN,NaN,47.0,"Amherst, MA",844.0,NaN,NaN,94,1004.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13521.0,NaN,NaN,...,678.0,NaN,NaN,92.0,"Barre, MA",3010.0,NaN,NaN,94,1005.0


In [23]:
# Reduce DataFrame
CombinedValues_reduced = CombinedValues
CombinedValues_reduced.drop(CombinedValues_reduced.columns[[0,1,2,3,4,
                                                            5,7,8,9,10,11,12,13,15,
                                                            16,17,18]], axis=1, inplace=True)
CombinedValues_reduced.head()

,ZIP,est,year,zip
0,NaN,439.0,94,1001.0
1,NaN,450.0,94,1002.0
2,NaN,10.0,94,1003.0
3,NaN,47.0,94,1004.0
4,NaN,92.0,94,1005.0


In [24]:
# Reset index for further data manipulation
cv_Final = CombinedValues_reduced.reset_index(drop = True)
cv_Final


,ZIP,est,year,zip
0,NaN,439.0,94,1001.0
1,NaN,450.0,94,1002.0
2,NaN,10.0,94,1003.0
3,NaN,47.0,94,1004.0
4,NaN,92.0,94,1005.0
5,NaN,182.0,94,1007.0
6,NaN,18.0,94,1008.0
7,NaN,14.0,94,1009.0
8,NaN,50.0,94,1010.0
9,NaN,16.0,94,1011.0


In [25]:
# Combine "zip" and "ZIP" columns into one, eliminating complementary NaN values
cv_Final['zipcode'] =  cv_Final['zip'].combine_first(cv_Final['ZIP'])
cv_Final

,ZIP,est,year,zip,zipcode
0,NaN,439.0,94,1001.0,1001.0
1,NaN,450.0,94,1002.0,1002.0
2,NaN,10.0,94,1003.0,1003.0
3,NaN,47.0,94,1004.0,1004.0
4,NaN,92.0,94,1005.0,1005.0
5,NaN,182.0,94,1007.0,1007.0
6,NaN,18.0,94,1008.0,1008.0
7,NaN,14.0,94,1009.0,1009.0
8,NaN,50.0,94,1010.0,1010.0
9,NaN,16.0,94,1011.0,1011.0


In [26]:
# Load NYC Zipcode GeoJson File into GeoPandas DataFrame
zip_shape = gpd.read_file("nyc-zip-code-tabulation-areas-polygons.geojson")
zip_shape.head()

,@id,BLDGpostalCode,CTY_FIPS,OBJECTID,PO_NAME,STATE,ST_FIPS,Shape_Area,Shape_Leng,borough,geometry,postalCode
0,http://nyc.pediacities.com/Resource/PostalCode...,0,081,1,Jackson Heights,NY,36,2.016328e+07,20624.692317,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372
1,http://nyc.pediacities.com/Resource/PostalCode...,0,081,2,Glen Oaks,NY,36,2.260653e+07,23002.816039,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004
2,http://nyc.pediacities.com/Resource/PostalCode...,0,081,3,New Hyde Park,NY,36,6.269333e+06,15749.161511,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040
3,http://nyc.pediacities.com/Resource/PostalCode...,0,081,4,Bellerose,NY,36,4.941836e+07,35932.810639,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426
4,http://nyc.pediacities.com/Resource/PostalCode...,0,081,5,Fresh Meadows,NY,36,6.938587e+07,38693.565676,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365


In [27]:
# Clean zipcode shapefile / eliminate columns
zip_shape = zip_shape.rename(columns={'postalCode': "zipcode"})
zip_shape = zip_shape[['zipcode', 'borough', 'geometry', 'PO_NAME']]
#datatype for all columns is 'object', changing zipcode to 'numeric'
zip_shape.zipcode = pd.to_numeric(zip_shape.zipcode)
zip_shape.head()

,zipcode,borough,geometry,PO_NAME
0,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights
1,11004,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,Glen Oaks
2,11040,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,New Hyde Park
3,11426,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,Bellerose
4,11365,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,Fresh Meadows


In [28]:
# Test for Duplicate Values
print (len(zip_shape))
print (len(zip_shape['zipcode'].unique()))

262
247


In [29]:
# Drop duplicates - code from: 
# http://stackoverflow.com/questions/22918212/fastest-way-to-drop-duplicated-index-in-a-pandas-dataframe
zip_shape[~zip_shape.zipcode.duplicated()]

,zipcode,borough,geometry,PO_NAME
0,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights
1,11004,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,Glen Oaks
2,11040,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,New Hyde Park
3,11426,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,Bellerose
4,11365,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,Fresh Meadows
5,11373,Queens,POLYGON ((-73.88722098467157 40.72752618766134...,Elmhurst
6,11001,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,Floral Park
7,11375,Queens,"POLYGON ((-73.8562474640478 40.7367227854824, ...",Forest Hills
8,11427,Queens,POLYGON ((-73.74168681424477 40.73682340115314...,Queens Village
9,11374,Queens,POLYGON ((-73.86451198956482 40.73407164841783...,Rego Park


In [43]:
#convert zipcode to numeric 
cv_Final["zipcode"] = cv_Final["zipcode"].apply(pd.to_numeric)

#merge shapefile and census information together on zipcode column
combinedtest = zip_shape.merge(cv_Final, on = "zipcode")
combinedtest.head()

,zipcode,borough,geometry,PO_NAME,ZIP,est,year,zip
0,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,1229.0,94,11372.0
1,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,1236.0,95,11372.0
2,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,1292.0,96,11372.0
3,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,1309.0,97,11372.0
4,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,11372.0,NaN,98,NaN


In [45]:
# Drop Unnecessary Columns from new, merged dataframe
merge_Reduct = combinedtest
merge_Reduct.drop(merge_Reduct.columns[[4,7]], axis=1, inplace=True)
merge_Reduct.head()

,zipcode,borough,geometry,PO_NAME,est,year
0,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1229.0,94
1,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1236.0,95
2,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1292.0,96
3,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1309.0,97
4,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,98


In [46]:
merge_Reduct

,zipcode,borough,geometry,PO_NAME,est,year
0,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1229.0,94
1,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1236.0,95
2,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1292.0,96
3,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1309.0,97
4,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,98
5,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,99
6,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,00
7,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,01
8,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,NaN,02
9,11372,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,Jackson Heights,1521.0,03
